# Assistants API - Knowledge Retrieval 

https://platform.openai.com/docs/assistants/tools/knowledge-retrieval

https://community.openai.com/t/new-assistants-api-a-potential-replacement-for-low-level-rag-style-content-generation/475677 

Watch:

https://youtu.be/5rcjGjgJNQc?t=600&si=d9OtX0nMi2Rv0fQV 

References:

https://community.openai.com/t/assistants-api-retrieval-pricing-how-much-does-this-cost/485188/8

https://medium.com/madhukarkumar/what-does-openais-announcement-mean-for-retrieval-augmented-generation-rag-and-vector-only-54bfc34cba2c

https://www.youtube.com/watch?v=ClfyQNkTeUc

https://www.pinecone.io/learn/assistants-api-canopy/




![Alt text](ret.jpg "Assistants")

![Alt text](objects.jpeg "Assistants_Objects")

https://cobusgreyling.medium.com/openai-assistant-with-retriever-tool-08e9158ca900 

In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv , find_dotenv

_ :  bool = load_dotenv(find_dotenv())

In [2]:
client : OpenAI = OpenAI()

### Knowledge Retrieval

Retrieval augments the Assistant with knowledge from outside its model, such as proprietary product information or documents provided by your users. Once a file is uploaded and passed to the Assistant, OpenAI will automatically chunk your documents, index and store the embeddings, and implement vector search to retrieve relevant content to answer user queries.

https://platform.openai.com/docs/assistants/tools/knowledge-retrieval 



### How it works

The model then decides when to retrieve content based on the user Messages. The Assistants API automatically chooses between two retrieval techniques:

1. it either passes the file content in the prompt for short documents, or
2. performs a vector search for longer documents

Retrieval currently optimizes for quality by adding all relevant content to the context of model calls. We plan to introduce other retrieval strategies to enable developers to choose a different tradeoff between retrieval quality and model usage cost.

https://platform.openai.com/docs/assistants/tools/how-it-works


### Step 1: Upload the file and Create an Assistant

In [3]:
from openai.types.beta import assistant

#Upload file with assistance purpose
file = client.files.create(
    file = open("Pdf.pdf" , "rb") , 
    purpose="assistants"
)
print(file)

FileObject(id='file-twEkhibRoWeEMiMzjybyTsCz', bytes=48802, created_at=1710714881, filename='Pdf.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [5]:
assistant = client.beta.assistants.create(
    name= "Student Support Assistant",
    instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.",
    model="gpt-3.5-turbo-1106",
    tools= [{"type ":"retrieval" },],
    file_ids=[file.id]
)

BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[0].type'. You provided 'type ', did you mean to provide 'type'?", 'type': 'invalid_request_error', 'param': 'tools[0].type', 'code': 'missing_required_parameter'}}

#### Step 02 :Create a Thread

In [51]:
from openai.types.beta import Thread

thread : Thread = client.beta.threads.create()

print(thread)

Thread(id='thread_nCThfdZkqc6nDX7Pj8pEvSRX', created_at=1706202732, metadata={}, object='thread')


#### Step 03 : Add Message to Thread

In [52]:
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When and which city Zia U. Khan was born?"
)


#### Step 04 : Run the Assistant

In [53]:
from openai.types.beta.threads.run import Run

run : Run = client.beta.threads.runs.create(

    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please adress the user as pakistani , This user is student of PIAIC"
)

#### Step 05 : Check the Run Status

In [54]:
run : Run  = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run)

Run(id='run_ceJO8mCfo97vbWGlNNsJIIzM', assistant_id='asst_jgsEDpblsyaN2SI5jJLNaQe9', cancelled_at=None, completed_at=None, created_at=1706202733, expires_at=1706203333, failed_at=None, file_ids=['file-gSxvfOTsUtR4YGUcQ4601tgW'], instructions='Please adress the user as pakistani , This user is student of PIAIC', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1706202733, status='in_progress', thread_id='thread_nCThfdZkqc6nDX7Pj8pEvSRX', tools=[ToolAssistantToolsCode(type='code_interpreter')], usage=None)


#### Step 06 : Display the Assitance Respone

In [55]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages: list[ThreadMessage] = client.beta.threads.messages.list(
  thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)


user: When and which city Zia U. Khan was born?
